# Seed Database

> Load Data Into DB

In [ ]:
import os
import pickle
import pandas as pd

from pathlib import Path
from sqlalchemy import create_engine
from sqlalchemy.sql import text
from spotify_etl.core import *
from dotenv import load_dotenv

In [ ]:
# | hide
pd.set_option("display.max_rows", 500)
pd.set_option("display.max_columns", 500)
pd.set_option("display.width", 1000)

In [ ]:
complete_history = get_pickle_data(Path("data/complete_history.pkl"))
track_df = get_pickle_data(Path("data/track_df.pkl"))
artist_df = get_pickle_data(Path("data/artist_df.pkl"))

In [ ]:
load_dotenv()
POSTGRES_URL = os.getenv("POSTGRES_URL")

In [ ]:
engine = create_engine(POSTGRES_URL)

## Creating Tables

> Creating Tables if not exists

To optimize query speed I'll be creating indexes on frequently filtered data

In [ ]:
def create_index(table, column):
    return text(
        f"CREATE INDEX IF NOT EXISTS idx_{table}_{column} ON {table}({column});"
    )

> Artist Metadata

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text(
            f"""
    CREATE TABLE IF NOT EXISTS artist_metadata (
        "artist_id" text PRIMARY KEY UNIQUE,
        "artist" text,
        "main_genre" text,
        "secondary_genre" text,
        "genres" text,
        "popularity" integer,
        "followers" integer,
        "image_xl" JSONB,
        "image_lg" JSONB,
        "image_md" JSONB,
        "image_sm" JSONB,
        "images" JSONB,
        "type" text,
        "uri" text,
        "external_url" text,
        "href" text       
    );
    """
        )
    )
    conn.commit()
    conn.execute(create_index("artist_metadata", "main_genre"))
    conn.execute(create_index("artist_metadata", "secondary_genre"))
    conn.commit()

> Track Metadata

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text(
            f"""
    CREATE TABLE IF NOT EXISTS track_metadata (
    "track_id" TEXT PRIMARY KEY UNIQUE,
    "song" TEXT,
    "album" TEXT,
    "explicit" BOOLEAN,
    "song_popularity" INTEGER,
    "main_artist_id" TEXT  REFERENCES artist_metadata(artist_id),
    "artist_names" TEXT,
    "artist_ids" TEXT,
    "album_id" TEXT,
    "album_release_date" TEXT,
    "album_release_date_precision" TEXT,
    "album_external_url" TEXT,
    "album_href" TEXT,
    "album_images" JSONB,
    "album_type" TEXT,
    "album_uri" TEXT,
    "danceability" FLOAT,
    "energy" FLOAT,
    "key" INTEGER,
    "loudness" FLOAT,
    "mode" INTEGER,
    "speechiness" FLOAT,
    "acousticness" FLOAT,
    "instrumentalness" FLOAT,
    "liveness" FLOAT,
    "valence" FLOAT,
    "tempo" FLOAT,
    "duration_ms" INTEGER,
    "time_signature" INTEGER,
    "album_release_year" INTEGER,
    "album_decade" TEXT,
    "image_lg" JSONB,
    "image_md" JSONB,
    "image_sm" JSONB,
    "analysis_url" TEXT
    );
    """
        )
    )
    conn.commit()
    conn.execute(create_index("track_metadata", "main_artist_id"))
    conn.execute(create_index("track_metadata", "album_id"))
    conn.execute(create_index("track_metadata", "album_release_year"))
    conn.execute(create_index("track_metadata", "album_decade"))
    conn.commit()

> Spotify History (Partition)

One of the main goals of this project is to analyze my listening history over time. With this in mind, I decided to partition the spotify history table by year to optimize for queries over specific periods of time. This approach allows queries to quickly access relevant data by narrowing down the search to specific partitions, reducing the need to scan the entire table. It's especially effective for managing and analyzing large volumes of data over defined time periods, making the system more responsive and scalable.

In addition to greatly enhancing query performance this method also simplifies maintenance tasks, such as data archiving or cleanup, by isolating data into manageable chunks.

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text(
            f"""
    -- Create the history table
    CREATE TABLE IF NOT EXISTS spotify_history (
        "id" INTEGER NOT NULL,
        "ts" TIMESTAMP NOT NULL,
        "username" INTEGER,
        "platform" text,
        "ms_played" INTEGER,
        "conn_country" text,
        "ip_addr_decrypted" text,
        "user_agent_decrypted" text,
        "song" text,
        "artist" text,
        "album" text,
        "URI" text,
        "reason_start" text,
        "reason_end" text,
        "shuffle" BOOLEAN,
        "skipped" text,
        "offline" text,
        "offline_timestamp" FLOAT,
        "incognito_mode" BOOLEAN,
        "month" smallint,
        "year" smallint,
        "track_id" text REFERENCES track_metadata(track_id),
        "main_artist_id" text REFERENCES artist_metadata(artist_id),
        "percent_played" FLOAT
    )  
    PARTITION BY RANGE (ts)
    ;
                      

    -- Create a sequence for the primary key
    CREATE SEQUENCE IF NOT EXISTS spotify_history_id_seq;
                      

    -- Create a function to create yearly partitions        
    CREATE OR REPLACE FUNCTION create_yearly_partitions(start_year INT, end_year INT)
    RETURNS VOID AS $$
    DECLARE
        current_year INT := start_year;
        partition_name TEXT;
        index_name TEXT;
    BEGIN
        WHILE current_year <= end_year LOOP
            partition_name := 'spotify_history_y' || current_year;
            index_name := partition_name || '_id_idx';

            -- Create partition table using dollar-quoting for the SQL string
            EXECUTE format($f$
                CREATE TABLE IF NOT EXISTS %I
                PARTITION OF spotify_history FOR VALUES FROM (%L) TO (%L);
            $f$, partition_name, current_year || '-01-01', (current_year + 1) || '-01-01');

            -- Create unique index on the partition table
            EXECUTE format($f$
                CREATE UNIQUE INDEX IF NOT EXISTS %I
                ON %I (id);
            $f$, index_name, partition_name);
    
            current_year := current_year + 1;
        END LOOP;
     END;
     $$ LANGUAGE plpgsql;
                      
    -- Add the yearly partitions
    SELECT create_yearly_partitions({complete_history.year.min()}, EXTRACT(YEAR FROM CURRENT_DATE)::INT);

    -- Create a function to automatically insert new rows into the correct partition
    CREATE OR REPLACE FUNCTION spotify_history_auto_id()
    RETURNS TRIGGER AS $$
    BEGIN
        -- Check if the new row's id is NULL or not provided and automatically assign a value from the sequence
        IF NEW.id IS NULL THEN
            NEW.id := nextval('spotify_history_id_seq');
        END IF;
        RETURN NEW;
    END;
    $$ LANGUAGE plpgsql;

    -- Trigger to automatically insert new rows into the correct partition
    CREATE OR REPLACE TRIGGER spotify_history_before_insert
    BEFORE INSERT ON spotify_history
    FOR EACH ROW EXECUTE FUNCTION spotify_history_auto_id();
    """
        )
    )
    conn.commit()
    conn.execute(create_index("spotify_history", "ts"))
    conn.execute(create_index("spotify_history", "main_artist_id"))
    conn.execute(create_index("spotify_history", "track_id"))
    conn.execute(create_index("spotify_history", "year"))
    conn.execute(create_index("spotify_history", "month"))
    conn.execute(create_index("spotify_history", "album"))
    conn.commit()

> Artist Tracks

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text(
            f"""
    CREATE TABLE IF NOT EXISTS artist_tracks (
        id SERIAL PRIMARY KEY UNIQUE,
        "track_id" TEXT REFERENCES track_metadata(track_id),
        "artist_id" TEXT REFERENCES artist_metadata(artist_id),
        "is_main_artist" BOOLEAN
    );
    """
        )
    )
    conn.commit()
    conn.execute(create_index("artist_tracks", "artist_id"))
    conn.execute(create_index("artist_tracks", "track_id"))
    conn.execute(create_index("artist_tracks", "is_main_artist"))
    conn.commit()

## Getting Data Ready

In [ ]:
# |code-fold: true
artist_metadata = artist_df[
    [
        "artist_id",
        "artist",
        "main_genre",
        "secondary_genre",
        "genres",
        "popularity",
        "followers",
        "image_xl",
        "image_lg",
        "image_md",
        "image_sm",
        "images",
        "type",
        "uri",
        "external_url",
        "href",
    ]
].copy()

track_metadata = track_df[
    [
        "track_id",
        "song",
        "album",
        "explicit",
        "song_popularity",
        "main_artist_id",
        "artist_names",
        "artist_ids",
        "album_id",
        "album_release_date",
        "album_release_date_precision",
        "album_external_url",
        "album_href",
        "album_images",
        "album_type",
        "album_uri",
        "danceability",
        "energy",
        "key",
        "loudness",
        "mode",
        "speechiness",
        "acousticness",
        "instrumentalness",
        "liveness",
        "valence",
        "tempo",
        "duration_ms",
        "time_signature",
        "album_release_year",
        "album_decade",
        "image_lg",
        "image_md",
        "image_sm",
        "analysis_url",
    ]
].copy()

spotify_history = complete_history[
    [
        "ts",
        "username",
        "platform",
        "ms_played",
        "conn_country",
        "ip_addr_decrypted",
        "user_agent_decrypted",
        "song",
        "artist",
        "album",
        "URI",
        "reason_start",
        "reason_end",
        "shuffle",
        "skipped",
        "offline",
        "offline_timestamp",
        "incognito_mode",
        "month",
        "year",
        "track_id",
        "main_artist_id",
        "percent_played",
    ]
].copy()

Creating a function to get the artist track data

In [ ]:
# |code-fold: true
def create_artist_track(trck_metadata_df):
    artist_tracks = (
        track_df[["track_id", "artist_ids", "main_artist_id"]]
        .reset_index(drop=True)
        .copy()
    )

    artist_tracks_exploded = artist_tracks.assign(
        artist_id=artist_tracks["artist_ids"].str.split(";;")
    ).explode("artist_id")

    artist_tracks_exploded["is_main_artist"] = (
        artist_tracks_exploded["artist_id"] == artist_tracks_exploded["main_artist_id"]
    )

    final_df = artist_tracks_exploded[[
        "track_id", "artist_id", "is_main_artist"]]

    return final_df.reset_index(drop=True)


artist_tracks = create_artist_track(track_df)

## Load Data

> If it hasn't already been loaded

Since the goal of this project is to work with a snapshot of the data, we can check if there's data in the tables before proceeding. 

In [ ]:
def insert_data(df, table):
    with engine.connect() as connection:
        result = connection.execute(
            text("SELECT COUNT(*) FROM spotify_history"))
        count = result.scalar()
        if count == 0:
            df.to_sql(table, connection, if_exists="append", index=False)
        else:
            print("Table already has data, skipping insert")

In [ ]:
insert_data(artist_metadata, "artist_metadata")

Table already has data, skipping insert


In [ ]:
insert_data(track_metadata, "track_metadata")

Table already has data, skipping insert


In [ ]:
insert_data(spotify_history, "spotify_history")

Table already has data, skipping insert


In [ ]:
insert_data(artist_tracks, "artist_tracks")

Table already has data, skipping insert


## Creating Materialized View

> ... Making Dashboarding Easier :)

Since the data is static and my is to build a dashboard I've chosen to write a materialized view. Using a materialized view is beneficial because it allows precomputing and storing complex joins and aggregations from the underlying data tables. This significantly enhances query performance when accessing the dashboard, as intensive computation is handled during the view's refresh phase rather than at query time. Since the data is static, the materialized view doesn't require frequent updates, making it an efficient way to provide quick access to processed data without extra load on the database during peak usage. This method is particularly effective for ensuring fast response times and improving scalability of the dashboard application.

In [ ]:
with engine.connect() as conn:
    conn.execute(
        text(
            f""" 
    CREATE MATERIALIZED VIEW IF NOT EXISTS spotify_data_overview AS
    with timePlayed AS (
	select
		sh.id,
		CAST(SUM(sh.ms_played) AS FLOAT) / (1000 * 60 * 60) as hours_played,
		CAST(SUM(sh.ms_played) AS FLOAT) / (1000 * 60) as minutes_played
	from 
		spotify_history as sh
	group by sh.id
)
    SELECT
        sh.id,
        sh.ts,
        tm.song,
        am.artist,
        tm.album,
        am.main_genre,
        am.secondary_genre,   
        am.genres as genre_list,
        tm.artist_names,
        tm.artist_ids,
        tm.image_lg as album_image_lg,
        tm.image_md as album_image_md,
        tm.image_sm as album_image_sm,
        tm.album_images,
        am.image_xl as artist_image_xl,
        am.image_lg as artist_image_lg,
        am.image_md as artist_image_md,
        am.image_sm as artist_image_sm,
        am.images as artist_images,
        tm.track_id,
        am.artist_id,
        tm.album_id,
        tm.explicit,
        tm.song_popularity,
        am.popularity AS artist_popularity,
        tm.danceability,
        tm.energy,
        tm.key,
        tm.loudness,
        tm.mode,
        tm.speechiness,
        tm.acousticness,
        tm.instrumentalness,
        tm.liveness,
        tm.valence,
        tm.tempo,
        tm.duration_ms,
        tm.time_signature,
        tm.album_release_date AS release_date,
        tm.album_release_year,
        tm.album_decade,   
        DATE_TRUNC('month', sh.ts) as "month", -- Truncate the timestamp to the month
        DATE_TRUNC('year', sh.ts) as "year", -- Truncate the timestamp to the year
        CASE
          WHEN am.main_genre ILIKE '% lo-fi' OR am.main_genre ILIKE 'lo-fi%' OR am.main_genre ILIKE '% lo-fi %' THEN 'Non-Rap'
          WHEN am.main_genre ILIKE '% hip hop' OR am.main_genre ILIKE 'hip hop%' OR am.main_genre ILIKE '% hip hop %'
              OR am.main_genre ILIKE '% rap' OR am.main_genre ILIKE 'rap%' OR am.main_genre ILIKE '% rap %' THEN 'Rap'
          ELSE 'Non-Rap'
        END AS genre_category,
        sh.ms_played,
		tp.minutes_played,
		tp.hours_played,
		DATE_TRUNC('day', sh.ts) as "day", -- Truncate the timestamp to the year
        sh.reason_start,
        sh.reason_end,
        sh.shuffle,
        CASE 
            WHEN tm.instrumentalness > 0.5 THEN TRUE
            ELSE FALSE
        END AS is_instrumental
    FROM spotify_history sh
    JOIN track_metadata tm ON sh.track_id = tm.track_id
    JOIN artist_metadata am ON sh.main_artist_id = am.artist_id
	JOIN timePlayed tp ON sh.id = tp.id
    """
        )
    )
    conn.commit()
    conn.execute(create_index("spotify_data_overview", "artist_id"))
    conn.execute(create_index("spotify_data_overview", "track_id"))
    conn.execute(create_index("spotify_data_overview", "album_id"))
    conn.execute(create_index("spotify_data_overview", "year"))
    conn.execute(create_index("spotify_data_overview", "month"))
    conn.execute(create_index("spotify_data_overview", "song"))
    conn.execute(create_index("spotify_data_overview", "artist"))
    conn.execute(create_index("spotify_data_overview", "album"))
    conn.execute(create_index("spotify_data_overview", "main_genre"))
    conn.execute(create_index("spotify_data_overview", "secondary_genre"))
    conn.execute(create_index("spotify_data_overview", "genre_list"))
    conn.execute(create_index("spotify_data_overview", "artist_names"))
    conn.execute(create_index("spotify_data_overview", "ts"))
    conn.execute(create_index("spotify_data_overview", "genre_category"))
    conn.commit()

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()